### Load tensorflow

In [1]:
import tensorflow as tf

In [2]:
tf.__version__

'1.14.0'

### Enable Eager Execution if you are using tensorflow 1.x

In [3]:
tf.enable_eager_execution()

### Collect Data

In [4]:
import pandas as pd

In [5]:
data = pd.read_csv('prices.csv')

### Check all columns in the dataset

In [6]:
data.head()

,date,symbol,open,close,low,high,volume
0,2016-01-05 00:00:00,WLTW,123.430000,125.839996,122.309998,126.250000,2163600.0
1,2016-01-06 00:00:00,WLTW,125.239998,119.980003,119.940002,125.540001,2386400.0
2,2016-01-07 00:00:00,WLTW,116.379997,114.949997,114.930000,119.739998,2489500.0
3,2016-01-08 00:00:00,WLTW,115.480003,116.620003,113.500000,117.440002,2006300.0
4,2016-01-11 00:00:00,WLTW,117.010002,114.970001,114.089996,117.330002,1408600.0


### Drop columns `date` and  `symbol`

In [10]:
data.drop(['date','symbol'], axis=1, inplace=True)

In [11]:
data.head()

,open,close,low,high,volume
0,123.430000,125.839996,122.309998,126.250000,2163600.0
1,125.239998,119.980003,119.940002,125.540001,2386400.0
2,116.379997,114.949997,114.930000,119.739998,2489500.0
3,115.480003,116.620003,113.500000,117.440002,2006300.0
4,117.010002,114.970001,114.089996,117.330002,1408600.0


### Consider only first 1000 rows in the dataset for building feature set and target set

In [12]:
data_filt = data.head(1000)

In [14]:
data_filt.shape

(1000, 5)

### Convert Float64 to Float32

In [15]:
data_filt.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 5 columns):
open      1000 non-null float64
close     1000 non-null float64
low       1000 non-null float64
high      1000 non-null float64
volume    1000 non-null float64
dtypes: float64(5)
memory usage: 39.1 KB


In [18]:
data_filt.open = data_filt.open.astype('float32') 
data_filt.close = data_filt.close.astype('float32') 
data_filt.low = data_filt.low.astype('float32') 
data_filt.high = data_filt.high.astype('float32') 
data_filt.volume = data_filt.volume.astype('float32')

C:\Users\VKE\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\generic.py:5096: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [19]:
data_filt.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 5 columns):
open      1000 non-null float32
close     1000 non-null float32
low       1000 non-null float32
high      1000 non-null float32
volume    1000 non-null float32
dtypes: float32(5)
memory usage: 19.6 KB


### Divide the data into train and test sets

In [32]:
data_filt.columns

Index(['open', 'close', 'low', 'high', 'volume'], dtype='object')

In [33]:
X_cols=['open', 'low', 'high', 'volume']
y_cols=['close']
X=data_filt[X_cols]
y=data_filt[y_cols]

In [34]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

In [35]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(700, 4)
(300, 4)
(700, 1)
(300, 1)


### Normalize Train and Test Data 

In [37]:
from sklearn.preprocessing import Normalizer
transformer = Normalizer()


In [ ]:
X_train_t = transformer.fit_transform(X_train)

In [39]:
X_test_t = transformer.fit_transform(X_test)

In [40]:
y_train_t = transformer.fit_transform(y_train)

In [41]:
y_test_t = transformer.fit_transform(y_test)

## Building the graph in tensorflow

2.Define Weights and Bias

In [42]:
#We are initializing weights and Bias with Zero
w = tf.zeros(shape=(4,1))
b = tf.zeros(shape=(1))

3.Prediction

In [43]:
def prediction(x, w, b):
    
    xw_matmul = tf.matmul(x, w)
    y = tf.add(xw_matmul, b)
    
    return y

4.Loss (Cost) Function [Mean square error]

In [44]:
def loss(y_actual, y_predicted):
    
    diff = y_actual - y_predicted
    sqr = tf.square(diff)
    avg = tf.reduce_mean(sqr)
    
    return avg

5.GradientDescent Optimizer to minimize Loss [GradientDescentOptimizer]

In [45]:
def train(x, y_actual, w, b, learning_rate=0.01):
    
    #Record mathematical operations on 'tape' to calculate loss
    with tf.GradientTape() as t:
        
        t.watch([w,b])
        
        current_prediction = prediction(x, w, b)
        current_loss = loss(y_actual, current_prediction)
    
    #Calculate Gradients for Loss with respect to Weights and Bias
    dw, db = t.gradient(current_loss,[w, b])
    
    #Update Weights and Bias
    w = w - learning_rate*dw
    b = b - learning_rate*db
    
    return w, b

## Execute the Graph for 100 epochs and observe the loss

In [46]:
#Train for 100 Steps
for i in range(100):
    
    w, b = train(X_train_t, y_train_t, w, b, learning_rate=0.01)
    print('Current Loss on iteration', i, 
          loss(y_train_t, prediction(X_train_t, w, b)).numpy())

Current Loss on iteration 0 0.9216003
Current Loss on iteration 1 0.8493483
Current Loss on iteration 2 0.78275895
Current Loss on iteration 3 0.7213897
Current Loss on iteration 4 0.66483295
Current Loss on iteration 5 0.61270916
Current Loss on iteration 6 0.56467384
Current Loss on iteration 7 0.5204017
Current Loss on iteration 8 0.4796038
Current Loss on iteration 9 0.44200194
Current Loss on iteration 10 0.4073489
Current Loss on iteration 11 0.37541276
Current Loss on iteration 12 0.34598136
Current Loss on iteration 13 0.3188555
Current Loss on iteration 14 0.2938576
Current Loss on iteration 15 0.27081957
Current Loss on iteration 16 0.2495871
Current Loss on iteration 17 0.230019
Current Loss on iteration 18 0.21198621
Current Loss on iteration 19 0.19536595
Current Loss on iteration 20 0.18004979
Current Loss on iteration 21 0.16593362
Current Loss on iteration 22 0.15292417
Current Loss on iteration 23 0.14093535
Current Loss on iteration 24 0.129886
Current Loss on iterati

In [47]:
#Loss decreased with iterations ..

### Get the shapes and values of W and b

In [48]:
#Check Weights and Bias
print('Weights:\n', w.numpy())
print('Bias:\n',b.numpy())

Weights:
 [[3.9902450e-05]
 [3.9542385e-05]
 [4.0231796e-05]
 [4.9156487e-01]]
Bias:
 [0.4915649]


### Linear Classification using Keras

### Building the simple Neural Network in Keras with one neuron in the dense hidden layer.
#### Use Mean square error as loss function and sgd as optimizer

In [49]:
#Initialize Sequential Graph (model)
model = tf.keras.Sequential()

#Add Dense layer for prediction - Keras declares weights and bias automatically
model.add(tf.keras.layers.Dense(1, input_shape=(4,)))

#Compile the model - add Loss and Gradient Descent optimizer
model.compile(optimizer='sgd', loss='mse')

### Execute the model

In [50]:
model.fit(X_train_t, y_train_t, epochs=100)

Epoch 1/100
700/700 [==============================] - 0s 125us/sample - loss: 0.9181
Epoch 2/100
700/700 [==============================] - 0s 23us/sample - loss: 0.1523
Epoch 3/100
700/700 [==============================] - 0s 26us/sample - loss: 0.0253
Epoch 4/100
700/700 [==============================] - 0s 31us/sample - loss: 0.0042
Epoch 5/100
700/700 [==============================] - 0s 31us/sample - loss: 6.9604e-04
Epoch 6/100
700/700 [==============================] - 0s 36us/sample - loss: 1.1552e-04
Epoch 7/100
700/700 [==============================] - 0s 33us/sample - loss: 1.9190e-05
Epoch 8/100
700/700 [==============================] - 0s 38us/sample - loss: 3.2012e-06
Epoch 9/100
700/700 [==============================] - 0s 23us/sample - loss: 5.4838e-07
Epoch 10/100
700/700 [==============================] - 0s 28us/sample - loss: 1.0865e-07
Epoch 11/100
700/700 [==============================] - 0s 31us/sample - loss: 3.5712e-08
Epoch 12/100
700/700 [============

Epoch 92/100
700/700 [==============================] - 0s 21us/sample - loss: 2.1264e-08
Epoch 93/100
700/700 [==============================] - 0s 20us/sample - loss: 2.1255e-08
Epoch 94/100
700/700 [==============================] - 0s 29us/sample - loss: 2.1270e-08
Epoch 95/100
700/700 [==============================] - 0s 28us/sample - loss: 2.1259e-08
Epoch 96/100
700/700 [==============================] - 0s 30us/sample - loss: 2.1261e-08
Epoch 97/100
700/700 [==============================] - 0s 21us/sample - loss: 2.1260e-08
Epoch 98/100
700/700 [==============================] - 0s 24us/sample - loss: 2.1249e-08
Epoch 99/100
700/700 [==============================] - 0s 23us/sample - loss: 2.1253e-08
Epoch 100/100
700/700 [==============================] - 0s 23us/sample - loss: 2.1245e-08


In [51]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 1)                 5         
Total params: 5
Trainable params: 5
Non-trainable params: 0
_________________________________________________________________


In [52]:
model.get_weights()

[array([[-0.35631853],
        [-0.6108549 ],
        [ 0.1064185 ],
        [ 0.31237453]], dtype=float32), array([0.6876928], dtype=float32)]

### Classification using Keras 

In [61]:
model.predict(X_test_t)

array([[1.0000598 ],
       [1.0000539 ],
       [1.0000622 ],
       [1.0000498 ],
       [1.0000651 ],
       [1.0000597 ],
       [0.9999671 ],
       [0.9999465 ],
       [1.0000621 ],
       [0.99999356],
       [1.0000576 ],
       [1.0000627 ],
       [1.0000565 ],
       [0.9999428 ],
       [1.0000486 ],
       [0.99987733],
       [1.0000664 ],
       [0.9999259 ],
       [0.9999021 ],
       [0.99986815],
       [0.99999243],
       [0.9992857 ],
       [1.0000614 ],
       [1.0000486 ],
       [1.0000632 ],
       [1.0000638 ],
       [0.99983025],
       [0.9998089 ],
       [1.0000665 ],
       [1.0000236 ],
       [1.0000594 ],
       [1.0000652 ],
       [1.0000224 ],
       [1.0000585 ],
       [0.9999945 ],
       [1.0000087 ],
       [1.0000665 ],
       [1.0000668 ],
       [1.0000505 ],
       [1.0000625 ],
       [0.9999504 ],
       [1.0000557 ],
       [1.0000621 ],
       [0.9999986 ],
       [1.0000603 ],
       [1.0000665 ],
       [0.99996376],
       [0.999

### Load the given Iris data using pandas (Iris.csv)

In [67]:
iris_data = pd.read_csv('iris.csv')

In [68]:
iris_data.shape

(150, 6)

In [69]:
iris_data.columns

Index(['Id', 'SepalLengthCm', 'SepalWidthCm', 'PetalLengthCm', 'PetalWidthCm',
       'Species'],
      dtype='object')

In [70]:
iris_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 6 columns):
Id               150 non-null int64
SepalLengthCm    150 non-null float64
SepalWidthCm     150 non-null float64
PetalLengthCm    150 non-null float64
PetalWidthCm     150 non-null float64
Species          150 non-null object
dtypes: float64(4), int64(1), object(1)
memory usage: 7.1+ KB


In [97]:
iris_data = iris_data.drop(['Id'], axis=1)

### Splitting the data into feature set and target set

In [99]:
X_i_cols=['SepalLengthCm', 'SepalWidthCm', 'PetalLengthCm', 'PetalWidthCm']
y_i_cols=['Species']
X_i=iris_data[X_i_cols]
y_i=iris_data[y_i_cols]


### Target set has different categories. So, Label encode them. And convert into one-hot vectors using get_dummies in pandas.

In [100]:
# import labelencoder
from sklearn.preprocessing import LabelEncoder

In [101]:
# instantiate labelencoder object
le = LabelEncoder()

In [102]:
# apply le on categorical feature columns
categorical_feature_mask = y_i.dtypes==object
categorical_cols = y_i.columns[categorical_feature_mask].tolist()

y_i[categorical_cols] = y_i[categorical_cols].apply(lambda col: le.fit_transform(col))


### Divide the dataset into Training and test (70:30)

In [103]:
X_i_train, X_i_test, y_i_train, y_i_test = train_test_split(X_i, y_i, test_size=0.3, random_state=0)

In [104]:
X_i_train.head()

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm
60,5.0,2.0,3.5,1.0
116,6.5,3.0,5.5,1.8
144,6.7,3.3,5.7,2.5
119,6.0,2.2,5.0,1.5
108,6.7,2.5,5.8,1.8


In [105]:
y_i_train.head()

,Species
60,1
116,2
144,2
119,2
108,2


In [106]:
y_i_train = y_i_train.to_numpy()

In [107]:
y_i_testy_i_test = y_i_test.to_numpy()

### Model
Build the model with following layers: <br>
1. First dense layer with 10 neurons with input shape 4 (according to the feature set) <br>
2. Second Dense layer with 8 neurons <br>
3. Output layer with 3 neurons with softmax activation (output layer, 3 neurons as we have 3 classes) <br>
4. Use SGD and categorical_crossentropy loss 

In [108]:
#Initialize Sequential model
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.BatchNormalization(input_shape=(4,)))

In [109]:
#First Dense Layer
model.add(tf.keras.layers.Dense(10))

In [110]:
#Add 1st hidden layer
model.add(tf.keras.layers.Dense(4, activation='sigmoid'))

In [111]:
#Add 2nd hidden layer
model.add(tf.keras.layers.Dense(8, activation='sigmoid'))

In [112]:
#Add OUTPUT layer
model.add(tf.keras.layers.Dense(3, activation='softmax'))

In [113]:
#Compile the model
model.compile(optimizer='sgd', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [114]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization_1 (Batch (None, 4)                 16        
_________________________________________________________________
dense_5 (Dense)              (None, 10)                50        
_________________________________________________________________
dense_6 (Dense)              (None, 4)                 44        
_________________________________________________________________
dense_7 (Dense)              (None, 8)                 40        
_________________________________________________________________
dense_8 (Dense)              (None, 3)                 27        
Total params: 177
Trainable params: 169
Non-trainable params: 8
_________________________________________________________________


### Fitting the model and predicting 

In [115]:
model.fit(X_i_train,y_i_train,          
          validation_data=(X_i_test,y_i_test),
          epochs=100,
          batch_size=32)

W0922 16:24:12.140506   220 deprecation.py:323] From C:\Users\VKE\AppData\Local\Continuum\anaconda3\lib\site-packages\tensorflow\python\ops\math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 105 samples, validate on 45 samples
Epoch 1/100
105/105 [==============================] - 1s 5ms/sample - loss: 1.2129 - acc: 0.3048 - val_loss: 1.1308 - val_acc: 0.4000
Epoch 2/100
105/105 [==============================] - 0s 285us/sample - loss: 1.1949 - acc: 0.3048 - val_loss: 1.1231 - val_acc: 0.4000
Epoch 3/100
105/105 [==============================] - 0s 199us/sample - loss: 1.1834 - acc: 0.3048 - val_loss: 1.1184 - val_acc: 0.4000
Epoch 4/100
105/105 [==============================] - 0s 171us/sample - loss: 1.1723 - acc: 0.3048 - val_loss: 1.1110 - val_acc: 0.4000
Epoch 5/100
105/105 [==============================] - 0s 104us/sample - loss: 1.1591 - acc: 0.3048 - val_loss: 1.1068 - val_acc: 0.4000
Epoch 6/100
105/105 [==============================] - 0s 142us/sample - loss: 1.1511 - acc: 0.3048 - val_loss: 1.1027 - val_acc: 0.4000
Epoch 7/100
105/105 [==============================] - 0s 133us/sample - loss: 1.1432 - acc: 0.3048 - val_loss: 1.0989 - val_acc: 0.400

105/105 [==============================] - 0s 76us/sample - loss: 1.0207 - acc: 0.6857 - val_loss: 1.0432 - val_acc: 0.6000
Epoch 61/100
105/105 [==============================] - 0s 171us/sample - loss: 1.0189 - acc: 0.6857 - val_loss: 1.0429 - val_acc: 0.6000
Epoch 62/100
105/105 [==============================] - 0s 161us/sample - loss: 1.0189 - acc: 0.6952 - val_loss: 1.0423 - val_acc: 0.6000
Epoch 63/100
105/105 [==============================] - 0s 95us/sample - loss: 1.0177 - acc: 0.6857 - val_loss: 1.0413 - val_acc: 0.6000
Epoch 64/100
105/105 [==============================] - 0s 161us/sample - loss: 1.0181 - acc: 0.6952 - val_loss: 1.0416 - val_acc: 0.6000
Epoch 65/100
105/105 [==============================] - 0s 171us/sample - loss: 1.0170 - acc: 0.6952 - val_loss: 1.0418 - val_acc: 0.6000
Epoch 66/100
105/105 [==============================] - 0s 76us/sample - loss: 1.0172 - acc: 0.6952 - val_loss: 1.0424 - val_acc: 0.6000
Epoch 67/100
105/105 [============================

### Report Accuracy of the predicted values

In [117]:
pred = model.predict(X_i_test)
pred

array([[0.2775474 , 0.30798203, 0.41447058],
       [0.32398248, 0.31586355, 0.36015403],
       [0.3864898 , 0.2937358 , 0.31977448],
       [0.26071197, 0.3164363 , 0.42285174],
       [0.39350513, 0.29431263, 0.31218222],
       [0.2670985 , 0.30853143, 0.4243701 ],
       [0.3937296 , 0.29348883, 0.3127816 ],
       [0.2766341 , 0.3123465 , 0.41101947],
       [0.276038  , 0.3149035 , 0.40905857],
       [0.30631366, 0.3116019 , 0.3820844 ],
       [0.28601155, 0.31462303, 0.39936548],
       [0.28479984, 0.310154  , 0.40504616],
       [0.3007831 , 0.31289852, 0.38631836],
       [0.28206185, 0.3133699 , 0.40456825],
       [0.29432   , 0.31156754, 0.39411247],
       [0.39453605, 0.29516816, 0.31029576],
       [0.29687476, 0.31050766, 0.39261752],
       [0.3289465 , 0.31034186, 0.3607116 ],
       [0.39461634, 0.29472405, 0.3106596 ],
       [0.38636935, 0.29411685, 0.31951377],
       [0.2915257 , 0.3082466 , 0.40022764],
       [0.31073675, 0.30769268, 0.38157055],
       [0.